# 1. 读写文本

## 1.1 使用 open() 读取

- rt 模式：读取
- wt 模式：写入
- at 模式：追加

```python
with open('somefile.txt', 'rt') as f:
    data = f.read()
    ...
```

```python
with open('somefile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)
    ...
```

## 1.2 使用 sys.getdefaultendoding() 查询默认的文本编码方式

```python
with open('somefile.txt', 'rt', encoding='latin-1') as f:
    ...
```

## 1.3 不使用 with 语句

with 语句会创建一个上下文环境（context），当程序的控制流程离开 with 语句块后，文件将自动关闭。

不使用 with 语句需要手动关闭。

```python
f = open('somefile.txt', 'rt')
data = f.read()
f.close()
```

## 1.4 换行符

在 UNIX 和 Windows 中换行符 \n 和 \r\n

Python 工作在“通用型换行符”模式下，会将换行符转换成 \n 字符；将 \n 转换成当前系统的默认换行符。如果不想要这种“翻译”，可以给 open() 函数提供 newline=''的参数

```python
f = open('hello.txt', 'rt')
f.read()
```

# 2. 输出重定向

为 print() 函数加上 file 关键字参数，可以将输出重定向到一个文件中

要确保文件是以文本模式打开，如果是以二进制模式打开，打印会失败

```python
with open('somefile.txt', 'rt') as f:
    print('Hello World!', filw=f)
```

# 3. 以不同的分隔符或行结尾符打印

修改分隔符或行结尾符

In [1]:
print('ACME', 50, 91.5, sep=',', end='!!\n')

ACME,50,91.5!!



使用 str.join() 也可以实现同样的效果，但 str.join() 只能处理字符串

In [2]:
row = ('ACME', 50, 91.5)
print(','.join(str(x) for x in row))

ACME,50,91.5


# 4. 读写二进制

## 4.1 直接读写

可以使用 rb 或 wb 模式对二进制数据进行读写

读取二进制数据时，所有的数据将以字节串（byte string）的形式返回。

同样地，当写入二进制数据时，数据必须以对象的形式来提供，且该对象可以将数据以字节形式暴露出来（即，字节串、bytearray对象等）

```python
with open('somefile.bin', 'rb') as f:
    data = f.read()
```

```python
with open('somefile.bin', 'wb') as f:
    f.write(b'Hello World!')
```

在做索引和迭代操作时，字符串会返回代表该字节的整数值而不是字符串

In [3]:
t = 'Hello World'
t[0]

'H'

In [4]:
b = b'Hello World'
b[0]

72

In [5]:
for c in b:
    print(c)

72
101
108
108
111
32
87
111
114
108
100


## 4.2 读写文本内容

如果在二进制文件中读取或写入文本内容，需要编码或解码操作

```python
with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')
```

```python
with open('somefile.bin', 'wb') as f:
    text = 'Hello World'
    f.write(text.encode('utf-8'))
```

## 4.3 读写内存

数组和 C 结构体这样的对象可以直接用来进行写操作，而不必将其转换为 byte 对象。这种行为可适用于任何实现了“缓冲区接口（buffer interface）”的对象，该接口直接将对象底层的内存缓冲区暴露给可以在其上进行的操作。

```python
import array
nums = array.array('i', [1, 2, 3, 4])
with open('data.bin', 'wb) as f:
          f.write(nums)
```

对于支持将二进制数据读入到它们的底层的内存中的对象，可以使用文件对象的 readinto() 方法。但这常常与平台特性相关，且可能依赖于字（word）的大小和字节序（即大端和小端）等属性。

```python
import array
a = array.array('i', [0, 0, 0, 0, 0, 0, 0, 0])
with open('data.bin', 'rb') as f:
    f.readinto(a)
```

# 5. 对不存在的文件执行写入操作

使用 open() 函数中 x 模式代替 w 模式（仅Python3）

如果文件是二进制的，用 xb 模式代替 xt  即可

```python
with open('somefile', 'wt') as f:
    f.write('Hello\n')
```

```python
with open('somefile', 'xt') as f:
    f.write('Hello\n')
```

# 6. 字符串 I/O 操作

将一段文本或二进制字符串写入类似文件的对象上

使用 io.StringIO() 和 io.BytesIO() 类创建类似于文件的对象

使用 s.write 和 print('', file= ) 写入

In [8]:
import io

In [13]:
s = io.StringIO()
s.write('Hello World\n')

12

In [14]:
print('This is a test', file=s)

使用 s.getvalue() 和 s.read() 读出

In [15]:
s.getvalue()

'Hello World\nThis is a test\n'

In [17]:
s = io.StringIO('Hello\nWorld\n')
s.read(4)

'Hell'

In [18]:
s.read()

'o\nWorld\n'

StringIO 和 BytesIO 是模拟一个普通文件，例如创建一个文件型对象，但五法工作在一个需要真正系统级文件的环境中

# 7. 读写压缩文件

读写使用 gzip 或 bz2 格式压缩过文件的数据

I/O 操作采用文本形式并执行 Unicode 编码和解码，二进制数据用 rb 或 wb

如果没有指定模式，默认使用二进制。支持 encoding, errors, newline 等关键字参数

In [19]:
import gzip
import bz2

将压缩文件以文本形式读出

```python
with gzip.open('somefile.gz', 'rt') as f:
    text = f.read()
```

```python
with bz2.open('somefile.bz2', 'rt') as f:
    text = f.read()
```

写入压缩数据

```python
with gzip.open('somefile.gz', 'wt') as f:
    f.write(text)
```

```python
with bz2.open('somefile.bz2', 'wt') as f:
    f.write(text)
```

压缩级别可以通过 compresslevel 指定，默认级别是 9

```python
with gzip.open('somefile.gz', 'wt', compresslevel=5) as f:
    f.write(text)
```

gzip.open() 和 bz2.open() 能够对已经以二进制模式打开的文件进行叠加操作

```python
f = open('somefile.gz', 'rb')
with gzip.open(f, 'rt') as g:
    text = g.read()
```

# 8. 对固定大小的记录进行迭代

使用 iter() 和 functools.partial() 对固定大小的记录或数据块进行迭代

In [21]:
from functools import partial

```python
RECORD_SIZE = 32
with open('somefile.data', 'rb') as f:
    records = iter(partial(f.read, RECORD_SIZE), b'')
    for r in records:
        ...
```

records 对象可迭代，会产生固定大小的数据块直到达到文件末尾（如果文件大小不是记录大小的整数倍，最后的数据块较小）

iter() 函数：如果传递一个可调用对象及一个哨兵值给它，可以创建一个迭代器。得到的迭代器会重复调用用户提供的可迭代对象，直到返回的值为哨兵值为止，此时迭代过程停止。b''在这里作哨兵值。

二进制文件最常见为用 functools.partial 函数；文本文件按行读取更普遍

# 9. 将二进制数据读取到可变缓冲区中

中间不经过任何拷贝环节，可以原地修改数据再写回文件中

使用 readinto()

In [22]:
import os.path

```python
def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf
```

readinto() 可以用来将数据填充到任何预分配好的数组中（包括 array 块或 numpy 创建的数组）

readinto() 是为已存在的缓存区填充内容，而不是分配新的对象，可以避免产生额外的内存分配

内存映像（memoryview）可以对已存在的缓冲区做切片处理，中间不涉及任何的拷贝操作，可以修改它的内容

In [44]:
buf = bytearray(b'Hello World')
m1 = memoryview(buf)
m2 = m1[-5:]
m2

使用 f.readinto() 需要检查它的返回值，即实际读取的字节数。如果字节数小于所提供的缓冲区大小，可能表示数据被截断或遭到了破坏

# 10. 对二进制文件做内存映射

通过内存映射的方式将一个二进制文件加载到可变的字节数组中，可以随机访问其内容，或者实现就地修改

使用 mmap 模块实现文件的内存映射

In [45]:
import os
import mmap

In [46]:
def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

要使用这个函数，需要准备一个已经创建好的文件并为之填充一些数据

```python
size = 1000000
with open('data', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')
```

In [48]:
m = memory_map('data')
len(m)

1000000

In [49]:
print(m[0: 10])
print(m[0])

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
0


In [50]:
m[0: 11] = b'Hello World'
m.close()

由 mmap() 返回的 mmap 对象也可以当做上下文管理器使用，在这种情况下，底层的文件会自动关闭

In [58]:
with open('data', 'rb') as f:
    print(f.read(11))
with memory_map('data') as m:
    print(len(m))
    print(m[0: 11])

b'Hello World'
1000000
b'Hello World'


In [60]:
m.closed

True

默认情况下，memory_map() 打开的文件既可以读也可以写，对数据的任何修改都会拷贝回原始的文件中

如果需要只读访问，可以为 access 参数提供 mmap.ACCESS_READ 值

``
m = memory_map(filename, mmap.ACCESS_READ)
``

如果只想在本地修改数据，并不想将这些修改写回到原始文件中，使用 mmap.ACCESS_COPY 参数

``
m = memory_map(filename, mmap.ACCESS_COPY)
``

与其打开文件后通过各种组合的 seek(), read(), write() 调用来访问，不如简单地将文件映射到内存，然后通过切片操作来访问数据

利用 memoryview 能够以不同的方式来解读数据

In [62]:
m = memory_map('data')
v = memoryview(m).cast('I')
v[0] = 7
m[0: 4]

b'\x07\x00\x00\x00'

In [63]:
m[0: 4] = b'\x07\x01\x00\x00'
v[0]

263

对某个文件进行内存映射并不会导致将整个文件读到内存中，操作系统只为文件保存一段虚拟内存

如果有多个 Python 解释器对同一个文件做了内存映射，得到的 mmap 对象可在解释器之间交换数据。有时可用这种方法作为通过管道或 socket 传输数据的代替方式

# 11. 处理路径名

In [64]:
import os

In [70]:
path = '/Users/zhenkai/data/data.csv'

路径的最后部分

In [71]:
os.path.basename(path)

'data.csv'

目录名（不包括最后的名字）

In [72]:
os.path.dirname(path)

'/Users/zhenkai/data'

路径名合并

In [73]:
os.path.join('tmp', 'data', os.path.basename(path))

'tmp\\data\\data.csv'

用户名路径

In [76]:
path = '~/Data/data.csv'
os.path.expanduser(path)

'C:\\Users\\gzk/Data/data.csv'

路径分割

In [77]:
os.path.splitext(path)

('~/Data/data', '.csv')

# 12. 检测文件是否存在

In [78]:
import os

## 12.1 检测是否存在

In [79]:
os.path.exists('/etc/passwd')

False

In [82]:
os.path.exists('C:\Windows')

True

## 12.2 检查文件类型

- os.path.isfile()

- os.path.isdir()

- os.path.islink()  (symbolic link)

- os.path.realpath()  (linked to)

In [85]:
os.path.isfile('C:\Windows')

False

In [86]:
os.path.isdir('C:\Windows')

False

## 12.3 得到元数据（文件大小或修改日期）

获取元数据时，需注意权限问题

```python
os.path.getsize('')
os.path.gettime('')
```

# 13. 获取目录内容的列表

In [90]:
import os

```python
names = os.listdir('somedir')
```

直接使用 listdir 会得到原始的目录文件列表，包括所有的文件、子目录、符号链接等

使用 os.path 模块筛选数据

```python
import os.path
names = [name for name in os.listdir('somedir') if os.path.isfile(os.path.join('somedir', name))]
dirnames = [name for name in os.listdir('somedir') if os.path.isdir(os.path.join('somedir', name))]
pyfiles = [name for name in os.listdir('somedir') if name.endswith('.py')]
```

文件名的匹配，可以使用 glob 或 fnmatch 模块

```python
import glob
pyfiles = glob.glob('somedir/*.py')
```

```python
from fnmatch import fnmatch
pyfiles = [name for name in os.listdir('somedir') if fnmatch(name, '*.py')]
```

使用 os.stat() 函数得到一些附加的元数据，如文件大小、修订日期等

```python
import os
import os.path
import glob

pyfiles = glob.glob('.*py')

name_sz_date = [(name, os.path.getsize(name), os.path.getmtime(name)) for name, size, mtime in name_sz_date:
    print(name, size, mtime)

file_metadate = [(name, os.stat(name)) for name in pyfiles]
for name, meta in file_matadate:
    print(name, meta.st_size, meta.st_mtime)
```

一般来说，os.listdir() 函数返回的条目会根据系统默认的文件名编码方式进行解码，但特定条件下会遇到无法解码的文件名

# 14. 绕过文件名编码

想要使用了原始文件名的文件执行 I/O 操作，这些文件名没有根据默认的文件名编码规则解码

默认情况根据 sys.getfilesystemencoding() 返回的编码格式进行编码和解码

In [93]:
import sys

In [94]:
sys.getfilesystemencoding()

'utf-8'

使用原始字节串指定文件名，对文件名进行处理

```python
import os

# unicode
with open('somefile\xf1o.txt', 'w') as f:
    f.write('Spicy!')

# directory listing (decode)
os.listdir('.')

# directory listing (raw)
os.listdir(b'.')

# open file with raw filename
with open(b'somefile\xcc\x83o.txt') as f:
    print(f.read())
```

# 15. 打印无法解码的文件名

当打印来路不明的文件名时，使用下面的方式避免错误

```python
def bad_filename(filename):
    return repr(filename)[1: -1]

try:
    print(filename)
except UnicodeEncodeError:
    print(bad_filename(filename))
```

Python 假设文件名都是根据 sys.getfilesystemencoding() 返回的编码方式进行编码，但有些文件系统不一定会强制执行这种编码约束

执行 os.listdir() 等命令，错误的文件名会使 Python 陷入窘迫境地，Python 既不能直接丢弃错误的名字，又不能转换为合适的字符串。为了解决这一问题，Python 会在文件名中取出一个无法解码的字节值 \xhh，将其映射为一个“代理编码（surrogate encoding）”中，代理编码由 Unicode 字符 \udchh 表示

可以以其他方式编码

```python
def bad_filename(filename):
    temp = filename.encode(sys.getfilesystemencoding(), errors='surrogateescape')
    return temp.decode('latin-1')
```

# 16. 为已打开的文件添加或修改编码方式

为一个二进制模式打开的文件添加 Unicode 编码/解码，可以用io.TextIOWrapper() 对象将其包装

```python
import urllib.request
import io

u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u, encoding='utf-8')
text = f.read()
```

想要修改一个以文本模式打开的文件的编码方式，可以在用新的编码前，用 detach() 方法将已有的文本编码层移除

```python
import sys
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')

# 查看现在的编码方式
sys.stdout.encoding
```

- io.TextIOWrapper 是一个文本处理层，负责编码和解码 Unicode
- io.BufferedWriter 是一个缓冲 I/O 层，负责处理二进制数据
- io.FileIO 是一个原始文件，代表操作系统底层的文件描述符

添加或修改文本的编码涉及添加或修改 io.TextIOWrapper 层；无法直接通过访问上面的属性来操纵不同的层次

detach() 方法将最上层的 io.TextIOWrapper 层同文件分离开来，并将下一个层次（io.BufferedWriter）返回。在这之后，最上层将不再起作用

# 17. 将字节数据写入文本文件

```python
import sys
sys.stdout.buffer.write(b'Hello\n')
```

# 18. 将已有的文件描述符包装为文件对象

文件描述符只是一个由操作系统分配的整数句柄，用来指代某种系统的 I/O 通道。如果刚好有一个这样的文件描述符，可以通过 open() 函数用 Python 文件对象对其进行包装，只需要将整数形式的文件描述符作为第一个参数取代文件名。

```python
import os

fd = os.open('somefile.txt', os.O_WRONLY | os.O_CREAT)
f = open(fd, 'wt')
f.write('hello world\n')
f.close()
```

当高层文件对象被关闭或销毁时，底层的文件描述符也会被关闭。如果不想要这种行为，只需给 open() 提供一个可选的 closefd=False 参数即可

```python
f = open(fd, 'wt', closefd=False)
```

在 UNIX 系统上，这种包装文件描述符的技术可以用来方便地对已不同方式打开的 I/O 通道（管道、socket 等）提供一个类似文件的接口。如果需要做到跨平台，应该直接使用 socket 的 makefile() 方法。

可以为一个已经打开的文件创建一种别名，使它的工作方式可以区别于首次打开的样子。

例如：创建一个文件对象，使它能够在 stdout 上产生出二进制数据（通常 stdout 是以文本模式打开的）

```python
import sys
bstdout = open(sys.stdout.fileno(), 'wb', closefd=False)
bstdout.write(b'Hello World\n')
bstdout.flush()
```

# 19. 创建临时文件和目录

创建一个未命名的临时文件

```python
from tempfile import TemporaryFile

with TemporaryFile('w+t') as f:
    # read/write
    f.write('Hello World\n')
    f.write('Testing\n')
    
    # seek back to begining
    f.seek(0)
    data = f.read()
    
# Temporary file is destroyed
```

或

```python
f = TemporaryFile('w+t')
...
f.close()
```

w+t 处理文本模式，w+b 处理二进制模式

TemporaryFile()可以接收和内建的 open() 函数一样的参数

```python
with TemporaryFile('w+t', encoding='utf-8', errors='ignore') as f:
    ...
```

在大多数 UNIX 系统中，TemporaryFile() 创建的文件都是未命名的，在目录中也没有对应的条目。如果想解除这种设置，可以使用NamedTemporaryFile()

```python
from tempfile import NamedTemporaryFile

with NamedTemporaryFile('w+t') as f:
    print('filename is:', f.name)
    ...
```

使用 TemporaryFile()，结果文件会在关闭时自动删除，如果不想删除，可以提供一个 delete=False 关键字参数

```python
with NamedTemporaryFile('w+t', delete=False) as f:
    print('filename is:', f.name)
    ...
```

创建一个临时目录

```python
from tempfile import TemporaryDirectory
with TemporaryDirectory() as dirname:
    print('dirname is:', dirname)
    ...
```

使用 mkstemp() 和 mkdtemp() 创建临时文件和目录，但这些函数不会进一步去处理文件管理的任务。例如，mkstemp() 只是简单地返回一个原始的操作系统文件描述符。转换为合适的文件和清理文件由我们自己完成

In [99]:
import tempfile

In [100]:
tempfile.mkstemp()

(19, 'C:\\Users\\gzk\\AppData\\Local\\Temp\\tmp6mw__xan')

In [101]:
tempfile.mkdtemp()

'C:\\Users\\gzk\\AppData\\Local\\Temp\\tmph0466xv3'

一般来说，临时文件都是在系统默认的区域中创建的，使用 tempfile.gettempdir() 找出实际位置

In [102]:
tempfile.gettempdir()

'C:\\Users\\gzk\\AppData\\Local\\Temp'

所有与临时文件有关的函数都可以使用 prefix, suffix, dir 等关键字参数

```python
f = NamedTemporaryFile(prefix='mytemp', suffix='.txt', dir='/tmp')
f.name
```

# 20. 同串口通信

使用 pySerial 包

```python
import serial

ser = serial.Serial('/dev/tty.usbmodem641', baudrate=9600, bytesize=8, parity='N', stopbits=1)
```

在 Windows 中，可以使用数字代表通信端口，如："COM0", "COM1"。一旦打开，可以通过 read(), readline(), write() 读写数据

# 21. 序列化 Python 对象

将 Python 对象序列化为字节流，就可以保存到文件中、存储到数据库中或通过网络连接传输

将某个对象转储到文件中

```python
import pickle

data = ...
f = open('somefiel', 'wb')
pickle.dump(data, f)
```

将对象转储为字符串

```python
s = pickle.dumps(data)
```

从字节流中重新创建对象

```python
f = open('somefile', 'rb')
data = pickle.load(f)
```

或

```python
data = pickle.loads(s)
```

序列化的数据中包含有每个对象的开始和结束以及有关对象类型的信息。对函数、类以及实例进行 pickle 处理，产生的数据只会对代码对象所关联的名称进行编码

当对数据做反序列化处理时，会假设所需的源文件都是可用的。这会成为一个潜在的维护性问题，因为所有的机器都必须能够访问到相同的源代码。因此决不能对非受信任的数据使用 pickle.load()。

对特定类型的对象无法进行 pickle 操作，这些对象一般都会涉及某种外部系统状态，比如打开的文件、打开的网络连接、线程、进程、栈帧等。用户自定义的类有时可通过提供 ``__getstate__()`` 和 ``__setstate__()`` 方法规避这些规则

对于大型的数据结构，如由 array 模块或 numpy 库创建的二进制数组，pickle 就不是一个特别高效的编码了，如果需要移动大量的数组型数据，最好将数据保存在文件中

由于 pickle 是 Python 的专有特性，且同源代码关联紧密，所以不应该把 pickle 作为长期存储的格式

下面这个类在内部定义了一个线程，但仍然可以进行 pickle/unpickle 操作

```python
import time
import threading

class Countdown:
    def __init__(self, n):
        self.n = n
        self.thr = threading.Thread(target=self.run)
        self.thr.daemon = True
        self.thr.start()
    
    def run(self):
        while self.n > 0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)
       
    def __getstate__(self):
        return self.n
    
    def __setstate__(self, n):
        self.__init__(n)
```